# Project - Day 4 - MLFlow training of your model

## Insert MLFlow parameters
The following cell is marked as `parameters`, you might find useful to include MLFlow usable parameters here for varying and experimenting different values for the CNN.

In [ ]:
#activation = 
#reg = 

## Excercise

Based on the Training step of the project done on day 3:

- train a model and store the metrics of the training process in MLFlow. e.g.:
```python
with mlflow.start_run(tags={"mlflow.runName": "train"}) as mlrun:

    losses = []
    val_losses = []
    !pip install -q tqdm
    from tqdm import trange
    
    n_epochs = 5
    n_blocks = y_train.numblocks[0]
    
    for epoch in trange(n_epochs):
        for X, y in zip(X_train.blocks, y_train.blocks):
            losses.append(
                (len(losses)/n_blocks, classifier.train_on_batch(X.compute(), y.compute()))
            )
        ls = classifier.test_on_batch(X_valid, y_valid)
        val_losses.append(
            (len(losses)/n_blocks,ls)
            )
        mlflow.log_metric("loss", ls, step=int(len(losses)/n_blocks))

```

- store the model in MLFlow of the usage on the next step of the pipeline, e.g.:

```python
    classifier.save("classifier.keras")
    mlflow.log_artifact("classifier.keras")
    prds = classifier.predict(X_valid.compute())
    signature = infer_signature(X_valid.compute(), prds)
    mlflow.tensorflow.log_model(classifier, "model", registered_model_name="CYGNO_CNN", signature=signature)
```

- store any additional plot that you find useful to track as a MLFlow artifact

In [ ]:
#Function definition
import numpy as np
import matplotlib.pyplot as plt
import dask, dask.array
import tensorflow as tf
from tqdm import trange 

## See Day 2
@dask.delayed 
def load_image(filename: str):
    """Wrapper function loading image as a dask.delayed"""
    return np.asarray(Image.open(filename))

## See Day 2
def load_raw_images(filenames):
    """Load the images from the file paths in `filenames` into a delayed dask-array"""
    return dask.array.stack([
        dask.array.from_delayed(load_image(f), shape=(576, 576), dtype=np.uint8) 
        for f in filenames
    ], axis=0)


## Discussed in Day 1, implemented in Day 2
def windowing(dask_image, x_min, x_max):
    """Maps the pixel values from the interval [x_min, x_max] to [0, 1]"""
    return dask.array.clip((dask_image - x_min)/(x_max - x_min), 0., 1.)

## Discussed in Day 1, implemented in Day 2
def crop_center(dask_image, half_win=90):        #64 standard
    """Crop a numpy-represented image around its center, the resulting image will be a square of side 2*half_win"""
    low, high = 576//2 - half_win, 576//2 + half_win
    return dask_image[:,low:high, low:high]

import re
def energy_keV_from_path(filenames):
    """
    Return a dask array with the energy (in keV) as obtained parsing a sequence of filenames passed 
    as an argument.
    """
    return dask.array.from_array([float(re.findall(r"/([0-9]+)_keV", f)[0]) for f in filenames])

def is_nuclear_from_path(filenames):
    """
    Return an array of boolean, true for nuclear recoil, or false for electron recoils as 
    obtained parsing the list of filenames passed as an argument.
    """
    return dask.array.from_array([float('NR' in re.findall(r"/([NE]R)/", f)) for f in filenames])

In [ ]:
#Code execution with preproc before ML part
from glob import glob


input_files = np.array(glob ("/home/jovyan/data/export/train/*/*/*.png"))
input_files = np.random.permutation(input_files)

forvalid=50
fortrain_input_files=input_files[forvalid:]
forvalid_input_files=input_files[:forvalid]

training_set = crop_center(windowing(load_raw_images(fortrain_input_files), 95, 120))
validation_set = crop_center(windowing(load_raw_images(forvalid_input_files), 95, 120))

training_label = is_nuclear_from_path(fortrain_input_files)
validation_label = is_nuclear_from_path(forvalid_input_files)

training_energy = energy_keV_from_path(fortrain_input_files)
validation_energy = energy_keV_from_path(forvalid_input_files)
print(training_set)

#Creation CNN
## The learning rate defines the leap taken at each update of the weights
learning_rate = 1e-3

## We create a deep sequential model (layers are executed in a sequence, one after the other)
classifier = tf.keras.models.Sequential([
    ## We reshape the images making it explicit that they have 1 single channel
    tf.keras.layers.Reshape((180, 180, 1), name="reshape"),      #ho fatto la half-win da 90

    ## First Convolutional block (conv + conv + max pooling)
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    
    ## Second Convolutional block (equal to the previous one)
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=2),

    ## Finally, we flatten the generated image and we use the computed channels as input of a 
    ## last dense layer, activated by a sigmoid.
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_normal'),
])

## Before training we may want to specify the input shape of the dataset:
classifier.build(input_shape=(None, 180, 180))

classifier.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), 
    optimizer=tf.keras.optimizers.Adam(learning_rate)
)

#Rechunking
batch_size = 30

X_train = dask.array.rechunk(training_set,(batch_size,180,180))
X_valid = dask.array.rechunk(validation_set,(batch_size,180,180))
y_train = dask.array.rechunk(training_label,(batch_size,180,180))
y_valid = dask.array.rechunk(validation_label,(batch_size,180,180))

In [ ]:
with mlflow.start_run(tags={"mlflow.runName": "train"}) as mlrun:

    losses = []
    val_losses = []
    !pip install -q tqdm
    from tqdm import trange
    
    n_epochs = 5
    n_blocks = y_train.numblocks[0]
    
    for epoch in trange(n_epochs):
        for X, y in zip(X_train.blocks, y_train.blocks):
            ls=classifier.train_on_batch(X.compute(), y.compute())
            losses.append( (len(losses)/n_blocks, ls) )
            mlflow.log_metric("loss", ls, step=int(len(losses)/n_blocks))

            
        val_ls = classifier.test_on_batch(X_valid, y_valid)
        val_losses.append(
            (len(losses)/n_blocks,val_ls)
            )
        mlflow.log_metric("val_loss", val_ls, step=int(len(losses)/n_blocks))

    predictions = np.concatenate([classifier(tf.constant(X)) for X in list(X_train.blocks) + list(X_valid.blocks)])
    labels = dask.array.concatenate([y_train, y_valid]).compute()
    plt.hist(predictions[labels>0.5], bins=np.linspace(0, 1, 51), histtype='step', hatch='/'*5, linewidth=2, label="Nuclear recoil")
    plt.hist(predictions[labels<0.5], bins=np.linspace(0, 1, 51), histtype='step', hatch='\\'*5, linewidth=2, label="Electronic recoil")
    plt.legend(title="Cygno-SIM")
    plt.xlabel("Classifier response")
    plt.ylabel("Events")
    plt.yscale('log')
    pathfig='outputs/response.png'
    plt.savefig(pathfig)

    mlflow.log_artifact(pathfig)

    classifier.save("classifier.keras")
    mlflow.log_artifact("classifier.keras")
    prds = classifier.predict(X_valid.compute())
    signature = infer_signature(X_valid.compute(), prds)
    mlflow.tensorflow.log_model(classifier, "model", registered_model_name="CYGNO_CNNgiodho", signature=signature)